In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
os.chdir('/Users/yiyujie/Desktop/program/Quantitative Asset Management')

## Q1
Construct the value-weighted market return using CRSP data,1 replicating the market return
time series available in Kenneth French website.2 Also calculate the equal-weighted market return,
and the lagged total market capitalization. Your output should be from January 1926 to December
2023, at a monthly frequency

In [16]:
def PS1_Q1(CRSP_stocks):
    CRSP_stocks['date'] = pd.to_datetime(CRSP_stocks['date'])
    CRSP_stocks['market_value'] = CRSP_stocks['PRC'] * CRSP_stocks['SHROUT'] * 1e-6
    CRSP_stocks['RET'] = np.where(CRSP_stocks['RET'].notna() & CRSP_stocks['DLRET'].notna(), 
                        (1 + CRSP_stocks['RET']) * (1 + CRSP_stocks['DLRET']) - 1, CRSP_stocks['RET'])
    CRSP_stocks['RET'] = np.where(CRSP_stocks['RET'].isna()  & CRSP_stocks['DLRET'].notna(), CRSP_stocks['DLRET'], CRSP_stocks['RET'])
    CRSP_stocks["lag_Mv"]= CRSP_stocks[["PERMNO", "market_value"]].groupby("PERMNO").shift(1)
    
    stock_ew_ret = CRSP_stocks.groupby([CRSP_stocks['date'].dt.year, CRSP_stocks['date'].dt.month])['RET'].mean()
    stock_vw_ret = CRSP_stocks.groupby([CRSP_stocks['date'].dt.year, CRSP_stocks['date'].dt.month]).apply(lambda x: (x['RET'] * x['lag_Mv']).sum() / x['lag_Mv'].sum())
    stock_lag_mv = CRSP_stocks.groupby([CRSP_stocks['date'].dt.year, CRSP_stocks['date'].dt.month])['lag_Mv'].sum()
    
    out_df = pd.DataFrame({
        'Year': stock_ew_ret.index.get_level_values(0),
        'Month': stock_ew_ret.index.get_level_values(1),
        'Stock_lag_MV': stock_lag_mv.values,
        'Stock_Ew_Ret': stock_ew_ret.values,
        'Stock_Vw_Ret': stock_vw_ret.values
    })
    return out_df


CRSP_stocks = pd.read_csv('crsp.csv')
# Share codes and exchage code that we'll use
CRSP_stocks = CRSP_stocks.loc[CRSP_stocks['EXCHCD'].isin([1,2,3])&CRSP_stocks['SHRCD'].isin([10,11,12])].reset_index(drop=True)
CRSP_stocks['DLRET'] = pd.to_numeric(CRSP_stocks['DLRET'], errors='coerce')
CRSP_stocks['RET'] = pd.to_numeric(CRSP_stocks['RET'], errors='coerce')
CRSP_stocks = CRSP_stocks.loc[CRSP_stocks['DLRET'].notna()|CRSP_stocks['RET'].notna()].reset_index(drop=True)

Monthly_CRSP_Stock = PS1_Q1(CRSP_stocks)
Monthly_CRSP_Stock

/var/folders/l_/m6g3497s5j77bqb9jr7506tc0000gn/T/ipykernel_97203/1706953456.py:10: RuntimeWarning: invalid value encountered in scalar divide
  stock_vw_ret = CRSP_stocks.groupby([CRSP_stocks['date'].dt.year, CRSP_stocks['date'].dt.month]).apply(lambda x: (x['RET'] * x['lag_Mv']).sum() / x['lag_Mv'].sum())


,Year,Month,Stock_lag_MV,Stock_Ew_Ret,Stock_Vw_Ret
0,1926,1,0.000000,0.023559,NaN
1,1926,2,22.937813,-0.053059,-0.030857
2,1926,3,23.566903,-0.096952,-0.063095
3,1926,4,23.531099,0.031756,0.038189
4,1926,5,21.612938,0.001973,0.014322
...,...,...,...,...,...
1171,2023,8,49669.382388,-0.067183,-0.019932
1172,2023,9,48514.792992,-0.067241,-0.047216
1173,2023,10,46318.641865,-0.081300,-0.029335
1174,2023,11,44661.831134,0.080359,0.093831


## Q2
Using the risk-free rate of return from French’s website3, report the following moments of the market excess returns for both time series (2 decimal digits): annualized return, annualized volatility, annualized Sharpe ratio, skewness, and excess kurtosis. You should be comparing between July 1926 to December 2023, at a monthly frequency

In [17]:
from scipy.stats import skew, kurtosis
import pandas_datareader

def PS1_Q2(Monthly_CRSP_Stock, FF_mkt):
    Monthly_CRSP_Stock = pd.merge(Monthly_CRSP_Stock,FF_mkt,on=['Year','Month'])
    Monthly_CRSP_Stock['excess_return'] = Monthly_CRSP_Stock['Stock_Vw_Ret'] - Monthly_CRSP_Stock['RF']
    
    annual_excess_return = Monthly_CRSP_Stock['excess_return'].mean() * 12
    annual_volatility = Monthly_CRSP_Stock['excess_return'].std() * np.sqrt(12)
    annual_sharpe_ratio = annual_excess_return / annual_volatility
    skewness = skew(Monthly_CRSP_Stock['excess_return'])
    excess_kurtosis = kurtosis(Monthly_CRSP_Stock['excess_return']) - 3

    ff_annual_excess_return = Monthly_CRSP_Stock['Mkt-RF'].mean() * 12
    ff_annual_volatility = Monthly_CRSP_Stock['Mkt-RF'].std() * np.sqrt(12)
    ff_annual_sharpe_ratio = ff_annual_excess_return / ff_annual_volatility
    ff_skewness = skew(Monthly_CRSP_Stock['Mkt-RF'])
    ff_excess_kurtosis = kurtosis(Monthly_CRSP_Stock['Mkt-RF']) - 3
    
    out_df = pd.DataFrame({
        'Estimated FF Market Excess Return': [annual_excess_return, annual_volatility, annual_sharpe_ratio, skewness, excess_kurtosis],
        'Actual FF Market Excess Return': [ff_annual_excess_return, ff_annual_volatility, ff_annual_sharpe_ratio, ff_skewness, ff_excess_kurtosis]
    }, index=['Annualized Mean', 'Annualized Std Dev', 'Annualized Sharpe Ratio', 'Skewness', 'Excess Kurtosis'])
    return out_df

FF_mkt = pandas_datareader.famafrench.FamaFrenchReader('F-F_Research_Data_Factors',start='1926-07',end='2024-02').read()[0]/100
FF_mkt = FF_mkt.reset_index()
FF_mkt['Year'] = FF_mkt['Date'].astype('str').apply(lambda x: int(x[:4]))
FF_mkt['Month'] = FF_mkt['Date'].astype('str').apply(lambda x: int(x[5:]))
FF_mkt[['Mkt-RF','SMB','HML','RF']] = FF_mkt[['Mkt-RF','SMB','HML','RF']].astype(float)
out_df = PS1_Q2(Monthly_CRSP_Stock, FF_mkt)
out_df

,Estimated FF Market Excess Return,Actual FF Market Excess Return
Annualized Mean,0.083660,0.081370
Annualized Std Dev,0.189273,0.185142
Annualized Sharpe Ratio,0.442007,0.439501
Skewness,0.239040,0.157456
Excess Kurtosis,5.117144,4.374877


## Q3
Report (up to 8 decimal digits) the correlation between your time series and French’s time
series, and the maximum absolute difference between the two time series. It is zero? If not, justify
whether the difference is economically negligible or not. What are the reasons a nonzero difference?
You should be comparing between July 1926 to December 2023, at a monthly freque

In [19]:
def PS1_Q3(Monthly_CRSP_Stock, FF_mkt):
    Monthly_CRSP_Stock = pd.merge(Monthly_CRSP_Stock,FF_mkt,on=['Year','Month'])
    Monthly_CRSP_Stock['excess_return'] = Monthly_CRSP_Stock['Stock_Vw_Ret'] - Monthly_CRSP_Stock['RF']
    
    out = (Monthly_CRSP_Stock[['excess_return','Mkt-RF']].corr().iloc[0,1],(Monthly_CRSP_Stock['excess_return'] - Monthly_CRSP_Stock['Mkt-RF']).max())
    return out

In [18]:
PS1_Q3(Monthly_CRSP_Stock, FF_mkt)

(0.9982347520975545, 0.027747149635267976)